# Developing JuliaFEM

Author(s): Jukka Aho

**Abstract**: Developer notes. In this notebook we give the general guidelines how to implement basic fundamentals to JuliaFEM. The notebook is rapidly updated and should always represent the newest "style" how to develop things to JuliaFEM. At this phase of project interfaces are changing rapidly. For this reason we aim to develop couple of simple functions which aim to test that elements, equations, solvers etc. work as expected, in the sense that their interface hasn't any design problems. (Yes, we do believe in testing and follow also other practices proven to be good, like continuous integration). The aim of this document is not to be the guide to "other contributors" but is more like an guide line for all contributors, including ourselves, how things should be done.

For demonstrational purposes we use the Poisson equation in our examples to demonstrate the main concepts of the package. Poisson equation has some analogies to solid mechanics. Consider for example $(EAu')' + q = 0$, which is nothing more than a Poisson equation defined in 1d.

Because this is the second important document in this source repository, any questions or suggestions araising from this notebook are very welcome and desirable and should be adressed to the our issue log: https://github.com/JuliaFEM/JuliaFEM.jl/issues. Easiest way to make world better is to register GitHub (takes 1 min) and drop an issue. And the most important document? It's the "how to contribute" guide, see https://github.com/JuliaFEM/JuliaFEM.jl/blob/master/CONTRIBUTING.rst.

In this notebook briefly we have defined the following "objects":

**Element**: object which holds basis functions and fields. One is able to interpolate fields using element. Introduction of new basis functions needs new elements. Typical elements: Lagrange elements, hierarchical elements, etc.

**Equation**: object which defines some field equation, like Poisson equation $\Delta u = f$, elasticity equation $\nabla \cdot \sigma = f$, etc., which needs to be solve. This is the physics we are trying to solve by approximating the equations in element area by some interpolation function provided by element.

**Problem**: object which maps equations to elements. For example, HeatProblem which maps Poisson equation to Lagrange elements or ElasticityProblem which maps elasticity equation to Lagrange elements.

**Solver**: object which takes one or more problems, solves them using some strategy (iterative methods, multigrid, direct methods, ...) and updates corresponding fields to elements. The is the section where any numerical crunching happens and is targeted to high-performance computing. 

These concepts are quite well separated so that development process can focus just one of thing of interest. In this notebook we will give instructions to all of the sections describing the whole development process from element level to global assembly and solution.

We have
- [x] **`test_element`**
- [ ] **`test_equation`**
- [ ] **`test_problem`** 
- [ ] **`test_solver`**

which can be used to test that implementation has all necessary things defined.


Poisson equation is used in this notebook to demonstrate all phases of development process. It's weak form is: find $u\in\mathcal{U}$ such that
\begin{equation}
    \int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x=\int_{\Omega}fv\,\mathrm{d}x+\int_{\Gamma_{\mathrm{N}}}gv\,\mathrm{d}s\quad\forall v\in\mathcal{V}.
\end{equation}

## Definitions / nomenclature

- weighted residual form: differential equation is multiplied by a weight function and integrating it.
- weak form, "principle of virtual work": typically established by partial integration of a weighted residual form. $\delta\mathcal{W}_{\mathrm{int}}=\delta\mathcal{W}_{\mathrm{ext}}$.
- variational form, "principle of minimum potential energy": there exists some functional  or "potential function" $\Pi$ we are minimizing

In [2]:
using Logging
using FactCheck
Logging.configure(level=DEBUG)

Logger(root,DEBUG,Base.PipeEndpoint(open, 0 bytes waiting),root)

## Developing own element

Element contains basis functions and one or several fieldsets.

Minimum requirements for element:
- subclass it from Element, unless want to implement everything by youself
- define basis and partial derivatives of it, because we need to interpolate over it
- give connectivity information, how this element is connected to other elements

Test the element using ``test_element`` function. It it passes, the element implementation should be fine.

In [4]:
using JuliaFEM: Element, Field, FieldSet, Basis

Here's the basic implementation for element:

In [5]:
type MyQuad4 <: Element
    connectivity :: Array{Int, 1}
    basis :: Basis
    fields :: Dict{Symbol, FieldSet}
end

Default constructor takes only connectivity information as input argument. Most important thing here is to define `Basis` which is used to interpolate fields.

In [6]:
function MyQuad4(connectivity)
    h(xi) = [
        (1-xi[1])*(1-xi[2])/4
        (1+xi[1])*(1-xi[2])/4
        (1+xi[1])*(1+xi[2])/4
        (1-xi[1])*(1+xi[2])/4]
    dh(xi) = [
        -(1-xi[2])/4.0   -(1-xi[1])/4.0
         (1-xi[2])/4.0   -(1+xi[1])/4.0
         (1+xi[2])/4.0    (1+xi[1])/4.0
        -(1+xi[2])/4.0    (1-xi[1])/4.0]
    basis = Basis(h, dh)
    MyQuad4(connectivity, basis, Dict())
end

MyQuad4

Also some basic information like number of basis functions and element dimension is needed.

In [7]:
JuliaFEM.get_number_of_basis_functions(el::Type{MyQuad4}) = 4
JuliaFEM.get_element_dimension(el::Type{MyQuad4}) = 2

get_element_dimension (generic function with 7 methods)

Next we check that everything is well defined:

In [8]:
using JuliaFEM: test_element
test_element(MyQuad4)

26-Oct 05:30:31:INFO:root:Testing element MyQuad4
26-Oct 05:30:31:INFO:root:number of basis functions in this element: 4
26-Oct 05:30:31:INFO:root:Initializing element
26-Oct 05:30:31:INFO:root:Element dimension: 2
26-Oct 05:30:31:INFO:root:Creating new scalar field JuliaFEM.Field{Array{Int64,1}}(0.0,0,[1,2,3,4])
26-Oct 05:30:31:INFO:root:basis at [0.0,0.0]: [0.25 0.25 0.25 0.25]
26-Oct 05:30:31:INFO:root:field val at [0.0,0.0]: 2.5
26-Oct 05:30:32:INFO:root:derivative of basis at [0.0,0.0]: [-0.5 0.5 0.5 -0.5
 -0.5 -0.5 0.5 0.5]
26-Oct 05:30:32:INFO:root:field val at [0.0,0.0]: [0.0 2.0]
26-Oct 05:30:32:INFO:root:Element MyQuad4 passed tests.


If `test_element` passes, elements *interface* should be well defined. After building element, one can interpolate things in it. Here we create three new fieldsets `temperature`, `geometry` and `heat coefficient`, add some values for them in time $t=0.0$ and $t=1.0$ and interpolate:

In [9]:
using JuliaFEM: FieldSet, Field
element = MyQuad4([1, 2, 3, 4])

geometry_field = Field(0.0, Vector[])  # Create empty field at time t=0.0
push!(geometry_field, [ 0.0, 0.0])  # push some values for field
push!(geometry_field, [ 1.0, 0.0])
push!(geometry_field, [ 1.0, 1.0])
push!(geometry_field, [ 0.0, 1.0])
geometry_fieldset = FieldSet("geometry")  # create fieldset "geometry"
push!(geometry_fieldset, geometry_field)  # add field to fieldset
push!(element, geometry_fieldset) # add fieldset to element

temperature_fieldset = FieldSet("temperature")
push!(temperature_fieldset, Field(0.0, [0.0, 0.0, 0.0, 0.0]))
push!(temperature_fieldset, Field(1.0, [1.0, 2.0, 3.0, 4.0]))
push!(element, temperature_fieldset)

displacement_fieldset = FieldSet("displacement")
push!(displacement_fieldset, Field(0.0, Vector[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]))
push!(displacement_fieldset, Field(1.0, Vector[[0.0, 0.0], [0.0, 0.0], [0.25, 0.0], [0.0, 0.0]]))
push!(element, displacement_fieldset)

heat_coefficient_fieldset = FieldSet("heat coefficient")
push!(heat_coefficient_fieldset, Field(0.0, 2))
push!(heat_coefficient_fieldset, Field(1.0, 3))
push!(element, heat_coefficient_fieldset)
element.fields

Dict{Symbol,JuliaFEM.FieldSet} with 4 entries:
  symbol("heat coefficien… => JuliaFEM.FieldSet(symbol("heat coefficient"),Juli…
  :geometry                => JuliaFEM.FieldSet(:geometry,JuliaFEM.Field[JuliaF…
  :temperature             => JuliaFEM.FieldSet(:temperature,JuliaFEM.Field[Jul…
  :displacement            => JuliaFEM.FieldSet(:displacement,JuliaFEM.Field[Ju…

In [10]:
# geometry midpoint of element
using JuliaFEM: get_basis, grad
basis = get_basis(element)
basis("geometry", [0.0, 0.0], 0.0)

2-element Array{Float64,1}:
 0.5
 0.5

In [11]:
# interpolate derivatives works too
dbasis = grad(basis)
dbasis("temperature", [0.0, 0.0], 0.5)  # temperature gradient at mid point of element at time t=0.5

1x2 Array{Float64,2}:
 0.0  1.0

In [12]:
# interpolating scalar -> scalar.
basis("heat coefficient", [0.0, 0.0], 0.5)

2.5

In [13]:
# set time to Inf to get very last field value and -Inf to get first one.
basis("heat coefficient", [0.0, 0.0], Inf)

3

In [14]:
# some continuum mechanics
∇u = dbasis("displacement", [0.0, 0.0], 1.0)
ɛ = 1/2*(∇u + ∇u')
Ω = 1/2*(∇u - ∇u')
X = basis("geometry", [0.0, 0.0], 1.0)
F = I + ∇u
C = F'*F
E = 1/2*(F'*F - I)  # Green-Lagrange strain tensor

2x2 Array{Float64,2}:
 0.132813   0.0703125
 0.0703125  0.0078125

Basically one could already write local stiffness matrices:

In [15]:
weights = [1.0, 1.0, 1.0, 1.0]
integration_points = 1.0/sqrt(3.0)*Vector[[-1, -1], [1, -1], [1, 1], [-1, 1]]

M = zeros(4, 4)
K = zeros(4, 4)
f = zeros(4)
k = 6
rho = 36
q = 4

for (w, xi) in zip(weights, integration_points)
    N = get_basis(element)
    ∇N = grad(N)  # gradient is with respect to "geometry" field
    detJ = det(N)(xi)
    M += w*rho*N(xi)'*N(xi)*detJ  # mass matrix
    K += w*k*∇N(xi)'*∇N(xi)*detJ  # stiffness matrix
    f += w*N(xi)'*q*detJ  # force vector
end
K, M, f

(
4x4 Array{Float64,2}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0,

4x4 Array{Float64,2}:
 4.0  2.0  1.0  2.0
 2.0  4.0  2.0  1.0
 1.0  2.0  4.0  2.0
 2.0  1.0  2.0  4.0,

4x1 Array{Float64,2}:
 1.0
 1.0
 1.0
 1.0)

but we have an higher level interface for that also.

### Summary of developing own elements

Element itself if not calculating anything but only stores fields and basis functions so that the fields can be interpolated. We will provide command `test_element` which will ensure that everything necessary is defined. While a lot of things needs to be defined, by subclassing from `Element` most of these are already defined, thanks to multiple dispatch.

## Developing own equation

Let's consider a Poisson equation
\begin{align}
-\nabla \cdot \left( k \nabla u\right) &= f && \text{on } \Omega \\
\frac{\partial u}{\partial n} &= g && \text{on } \Gamma_{\mathrm{N}} \\
u &= u_0 && \text{on } \Gamma_{\mathrm{D}} \\
\end{align}

Weak form is, find $u\in\mathcal{U}$ such that
\begin{equation}
    \int_{\Omega}k \nabla u\cdot\nabla v\,\mathrm{d}x = \int_{\Omega}fv\,\mathrm{d}x + \int_{\Gamma_{\mathrm{N}}}gv\,\mathrm{d}s \quad\forall v\in\mathcal{V}.
\end{equation}

Minimum requirements for equation: 
- subclass from Equation, if not want to implement from scratch
- provide the name of the unknown field variable trying to solve
- default constructor takes the element as input argument

This time we will have function `test_equation`, which we can use to test that everything is working as expected. 

In [16]:
using JuliaFEM: Equation, IntegrationPoint, Quad4, Seg2, get_unknown_field_name

abstract Heat <: Equation
JuliaFEM.get_unknown_field_name(eq::Heat) = symbol("temperature")

get_unknown_field_name (generic function with 4 methods)

Basic data type needs to contain element, integration points and global degrees of freedom for global assembly:

In [17]:
""" Diffusive heat transfer for 4-node bilinear element. """
type DC2D4 <: Heat
    element :: Quad4
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end
function DC2D4(element::Quad4)
    integration_points = [
        IntegrationPoint(1.0/sqrt(3.0)*[-1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1,  1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[-1,  1], 1.0)]
    push!(element, FieldSet("temperature"))
    DC2D4(element, integration_points, [])
end
Base.size(equation::DC2D4) = 4

""" Diffusive heat transfer for 2-node linear segment. """
type DC2D2 <: Heat
    element :: Seg2
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end
function DC2D2(element::Seg2)
    integration_points = [
        IntegrationPoint([0.0], 2.0)]
    push!(element, FieldSet("temperature"))
    DC2D2(element, integration_points, [])
end
Base.size(equation::DC2D2) = 2

size (generic function with 64 methods)

Now the actual implementation for $\int_{\Omega}k \nabla u\cdot\nabla v\,\mathrm{d}x = \int_{\Omega}fv\,\mathrm{d}x + \int_{\Gamma_{\mathrm{N}}}gv\,\mathrm{d}s$.

The low-level way is to write equations for local matrix representation directly:

In [18]:
using JuliaFEM: get_basis, get_element

""" Mass matrix for dynamical problems. Not used in this example. """
function JuliaFEM.get_mass_matrix(equation::DC2D4, ip, time)
    element = get_element(equation)
    basis = get_basis(element)
    ρ = basis("density", ip, time)
    return ρ * basis(ip,time)'*basis(ip,time)
end
""" Left hand side defined in integration point. """
function JuliaFEM.get_stiffness_matrix(equation::DC2D4, ip, time)
    element = get_element(equation)
    basis = get_basis(element)
    dbasis = grad(basis)
    k = basis("temperature thermal conductivity", ip, time)
    return k * dbasis(ip,time)'*dbasis(ip,time)
end
""" Right hand side defined in integration point. """
function JuliaFEM.get_force_vector(equation::DC2D4, ip, time)
    element = get_element(equation)
    basis = get_basis(element)
    f = basis("temperature load", ip, time)
    return basis(ip,time)'*f
end
JuliaFEM.has_mass_matrix(equation::DC2D4) = true
JuliaFEM.has_stiffness_matrix(equation::DC2D4) = true
JuliaFEM.has_force_vector(equation::DC2D4) = true

""" Right hand side defined in integration point. """
function JuliaFEM.get_force_vector(equation::DC2D2, ip, time)
    element = get_element(equation)
    basis = get_basis(element)
    g = basis("temperature flux", ip, time)
    return basis(ip,time)'*g
end
JuliaFEM.has_force_vector(equation::DC2D2) = true

has_force_vector (generic function with 3 methods)

It might at start to feel a bit ackward to pass integration point and time all around as function arguments, so a little reasoning why this is essential might be needed.

First of all, equations are written in integration or gauss quadrature points for a certain reason. The reason is that for some incremental constitutive models internal variables are needed, which are saved to FieldSets inside integration points. We can always access these variables through `ip.fields` in the same way we can access all other field variables defined inside element via `element.fields`.

Secondly, starting point for our equation is that everything can be time-dependent. FieldSet is, like name suggests, a set of fields with different times and/or increments. By transferring time parameter inside equation makes it possible to access not only current field values but earlier values too. In this example field variables thermal conductivity $k$, load $f$ and flux $g$ are constant in time but this way they can depend easily from spatial or temporal dimension. Actually let's change the parameter $g$ such a way that it is linear ramp from $g(t) = 6t$.

If we want to try out this formulation, we must create element and assign this equation for it:

In [19]:
# create volume element
element = Quad4([1, 2, 3, 4])
fieldset1 = FieldSet("geometry", [Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]])])
fieldset2 = FieldSet("temperature thermal conductivity", [Field(0.0, 6.0)])
fieldset3 = FieldSet("temperature load", [Field(0.0, [12.0, 12.0, 12.0, 12.0])])
fieldset4 = FieldSet("density", [Field(0.0, 1.0)])
push!(element, fieldset1)
push!(element, fieldset2)
push!(element, fieldset3)
push!(element, fieldset4)
equation = DC2D4(element)

# create boundary element with 
boundary_element = Seg2([1, 2])
push!(boundary_element, FieldSet("geometry", [Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0]])]))
# linear ramp from 1 to 6 in time 0 to 1
push!(boundary_element, FieldSet("temperature flux", [Field(0.0, 0.0), Field(1.0, 6.0)]))
boundary_equation = DC2D2(boundary_element);

In [20]:
using JuliaFEM: initialize_local_assembly, calculate_local_assembly!

local_assembly = initialize_local_assembly(equation)
calculate_local_assembly!(local_assembly, equation)
local_assembly.stiffness_matrix

4x4 Array{Float64,2}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

In [21]:
local_assembly.force_vector

4x1 Array{Float64,2}:
 3.0
 3.0
 3.0
 3.0

Let's set homogeneous Dirichlet boundary condition ($u=0$) on boundary on $\Gamma_\mathrm{D} = \{(x,y) | 0 \leq x \leq 1, y=1\}$. With source term $f=12$ and $k=6$ we should have constant $T=1$ on free boundary. According to my math we should have $u\left(x,y\right)=-\frac{1}{6}\left(\frac{1}{2}fx^{2}-fx\right)$ which equals to one if $x=1$.

In [22]:
A = local_assembly.stiffness_matrix
b = local_assembly.force_vector
u = zeros(4)
fdofs = [1,2]
u[fdofs] = A[fdofs, fdofs] \ b[fdofs]

2-element Array{Float64,1}:
 1.0
 1.0

Notice that we created a separate boundary element which integrates heat flux. Defining boundary elements goes identically with other elements. With flux $g=6$ we should have constant $T=1$ on free boundary. This should equal to $u(x,y) = x$, i.e. 1d bar stretched from other side while other one is fixed.

In [23]:
boundary_assembly = JuliaFEM.initialize_local_assembly(boundary_equation)
JuliaFEM.calculate_local_assembly!(boundary_assembly, boundary_equation)
b = zeros(4)
b[fdofs] = boundary_assembly.force_vector
u = zeros(4)
u[fdofs] = A[fdofs, fdofs] \ b[fdofs]

2-element Array{Float64,1}:
 1.0
 1.0

### Some alternative ways to define equations - principle of minimum potential energy

Fundamentally we are always looking for a solution of systems of equation $Ax = b$, despite the fact that mathematicians doesn't like this "engineering approach" at all. For some equations it may be too hard to write left hand side, i.e., stiffness matrix $A$. (At least for me). For this reason JuliaFEM supports automatic differentiation using package called `ForwardDiff`, which makes the analytical linearization unnecessary.

Let's consider the following functional
\begin{equation}
\min\, J\left(u\right)=\int_{\Omega}(k+6u)\left|\nabla u\right|^{2}\,\mathrm{d}x-Pu,
\end{equation}
where $P$ contains point loads at the free corners of the domain. This is nothing more but a redefinition of the earlier example problem defined such that the source term $k$, which in previous example was constant, is no more constant, but is replaced a term which depend from field $u$: $q(u) = k + 6u$.

This functional doesn't actually have any or very little physical meaning, but it's non-linear due to the source term $q(u) = k+6u$ and therefore suits to our needs in demonstration purposes. In mechanical world this could be some sort of elastic spring with a spring "constant" depending on displacement (non-linear elasticity problem), which is compressed using point force $P$. The system is still *conservative*, i.e. it has a potential energy and can be solved using standard variational methods, finding a minimum of potential energy in suitable finite set of functions.

In JuliaFEM we simply define it's potential energy functional $\Pi(u)$ and let the `ForwardDiff`-package to handle the rest of the hard things. Because the problem is nonlinear, linearization and iterative solution method (=Newton's method) is needed. Accurate solution at the free end is
\begin{equation}
u = \frac{2}{3}.
\end{equation}

In [24]:
""" Diffusive heat transfer for 4-node bilinear element, with a nonlinear source term. """
type DC2D4NL <: Heat
    element :: Quad4
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end
function DC2D4NL(element::Quad4)
    integration_points = [
        IntegrationPoint(1.0/sqrt(3.0)*[-1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1,  1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[-1,  1], 1.0)]
    push!(element, FieldSet("temperature"))
    # Initial configuration needs to be defined
    push!(element["temperature"], Field(0.0, [0.0, 0.0, 0.0, 0.0]))
    DC2D4NL(element, integration_points, [])
end
Base.size(equation::DC2D4NL) = 4

size (generic function with 65 methods)

This time we don't write stiffness matrix, etc. explicitly but just write potential energy and let `ForwardDiff` take care of rest of the stuff.

In [32]:
""" Calculate a potential Π = Wint - Wext of system. """
function JuliaFEM.get_potential_energy(equation::DC2D4NL, ip, time; variation=nothing)
    element = get_element(equation)
    basis = get_basis(element)
    k = basis("temperature thermal conductivity", ip, time)
    f = basis("temperature load", ip, time)
    T = basis("temperature", ip, time, variation)
    ∇T = grad(basis)("temperature", ip, time, variation)
    Wint = (k + 6*T) * 1/2*∇T*∇T'
    Wext = f*T
    return Wint - Wext
end
JuliaFEM.has_potential_energy(eq::DC2D4NL) = true

has_potential_energy (generic function with 2 methods)

Solving non-linear variational problem is rather complicated thing due to the nature of non-linearity. Here's a heavily commented version of simple Newton iteration.

In [34]:
function run_simulation()
    # create model -- start
    element = Quad4([1, 2, 3, 4])
    fieldset1 = FieldSet("geometry")
    field1 = Field(0.0, Vector[[0.0,0.0], [1.0,0.0], [1.0,1.0], [0.0,1.0]])
    push!(fieldset1, field1)
    fieldset2 = FieldSet("temperature thermal conductivity")
    push!(fieldset2, Field(0.0, 6.0))
    fieldset3 = FieldSet("temperature load")
    push!(fieldset3, Field(0.0, 0*[12.0, 12.0, 12.0, 12.0]))
    fieldset4 = FieldSet("temperature nodal load")
    push!(fieldset4, Field(0.0, [3.0, 3.0, 0.0, 0.0]))  # <-- P is defined here
    push!(element, fieldset1)
    push!(element, fieldset2)
    push!(element, fieldset3)
    push!(element, fieldset4)
    equation = DC2D4NL(element)
    # create model -- end

    T0 = element["temperature"][1]  # initial temperature field, we need something to "variate"
    la = initialize_local_assembly(equation) # create workspace for local matrices
    T = zeros(4) # create workspace for solution vector
    ΔT = zeros(4) # 
    fd = [1, 2]
    tic()
    # start loops, in principle solve ∂r(u)/∂uΔu = -r(u) and update.
    for i=1:5
        la = initialize_local_assembly(equation,la)  # empty workspace -- every iteration should start with this
        calculate_local_assembly!(la, equation)  # calculate local matrices
        ΔT[fd] = la.stiffness_matrix[fd,fd] \ la.force_vector[fd]  # <-- note sign convention, more on this below
        T = T + ΔT  # add increment to previous value
        # create a new field "similar" to field T0 (i.e., same dimension of field variable with new data)
        new_field = similar(T0, T)
        new_field.time = 1.0
        new_field.increment = i
        push!(element["temperature"], new_field)  # add new field to "temperature" fieldset of element
        # print some convergence information
        @printf("increment %2d, |du| = %8.5f, |r| = %8.3f\n", i, norm(ΔT), norm(b[fd]))
    end
    toc()
    err = element["temperature"][end].values[1] - 2/3
    println("error: $err")
    blaa = element["temperature"][end].values[1:2]
    acc = 2/3
    println("temperature at free end: $blaa, should be $acc")
end
run_simulation()

increment  1, |du| =  1.41421, |r| =    4.243
increment  2, |du| =  0.42426, |r| =    4.243
increment  3, |du| =  0.04657, |r| =    4.243
increment  4, |du| =  0.00057, |r| =    4.243
increment  5, |du| =  0.00000, |r| =    4.243
elapsed time: 0.001905274 seconds
error: 1.6653345369377348e-15
temperature at free end: [0.6666666666666683,0.6666666666666683], should be 0.6666666666666666


#### A short note about sign convention

When solving a non-linear equations like $\mathbf{r}\left(\mathbf{u}\right) = 0$ it is necessary to iterate, i.e. solve several linearized solutions using e.g. Newton iterations until residual is small enough. Using truncated Taylor series the solution algorithm is in general
\begin{equation}
\mathbf{r}\left(\mathbf{u}+\Delta\mathbf{u}\right)=\mathbf{r}\left(\mathbf{u}\right)+\frac{\partial\mathbf{r}\left(\mathbf{u}\right)}{\partial\mathbf{u}}\Delta\mathbf{u}+\cdots\approx\mathbf{r}\left(\mathbf{u}\right)+\mathbf{K}\Delta\mathbf{u}=0
\end{equation}
\begin{equation}
\mathbf{K}\Delta\mathbf{u}=-\mathbf{r}\left(\mathbf{u}\right)
\end{equation}
with a *negative* sign in front of residual vector.

We do however want to solve something like
\begin{equation}
\mathbf{A}\Delta\mathbf{x}^{\left(i+1\right)}=\mathbf{b},\qquad\mathbf{x}^{\left(i+1\right)}=\mathbf{x}^{\left(i\right)}+\Delta\mathbf{x}^{\left(i+1\right)},
\end{equation}
where new increment is *added* to the old result. If this doesn't say anything, compare the equation to the 1d Newton iteration (https://en.wikipedia.org/wiki/Newton's_method)
\begin{equation}
x_{n+1}=x_{n}-\frac{f\left(x_{n}\right)}{f'\left(x_{n}\right)}
\end{equation}

Our convention is that the negative sign of residual vector is positive. It makes sense because this way the "normal" linear system force vector is like it's used to be in school books. Typically the residual vector is defined $\mathbf{r}=\mathbf{f}_{\mathrm{int}}-\mathbf{f}_{\mathrm{ext}}=0$. In our case the residual vector must be multiplied by $-1$ after the linearization so that the increment can be *added* to old result. In some text books this has already be taken into account by defining right hand side as $\mathbf{r}=\mathbf{f}_{\mathrm{ext}}-\mathbf{f}_{\mathrm{int}}$ while linearization is still done for vector $\mathbf{r}=\mathbf{f}_{\mathrm{int}}-\mathbf{f}_{\mathrm{ext}}$. Quite confusing!

Just keep in mind that there is always minus somewhere and it should be. In our variational implementation the sign of residual vector is automatically changed so that next increment is  $\mathbf{A} \Delta\mathbf{x}=\mathbf{b}$. Like in the simple example above.

### Principle of virtual work

Defining variational form or "energy form", or in other words, using principle of minimum potential energy, requires system to be *conservative*, i.e. there is some energy functional describing the system. Indeed it's a very elegant approach to model e.g. hyperelasticity and other systems without a loss of energy. When system has energy dissipation, principle of minimum potential energy cannot be used for obvious reasons. There is no any "potential function" $\Pi$ which could be defined and variated around it's equilibrium state. Possible situations when this happens include e.g. material plasticity or friction. In these situations the principle of virtual work is useful.

In this situation we find equation $\mathbf{r}=\mathbf{f}_{\mathrm{int}}-\mathbf{f}_{\mathrm{ext}}=0$ which needs to be solved. Again we let `ForwardDiff` to do the linearization of the right hand side. Let's demonstrate this too.

In [37]:
using JuliaFEM: get_field

""" Diffusive heat transfer for 4-node bilinear element,
with a nonlinear term to be added later.. """
type DC2D4NLY <: Heat
    element :: Quad4
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end
function DC2D4NLY(element::Quad4)
    integration_points = [
        IntegrationPoint(1.0/sqrt(3.0)*[-1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1,  1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[-1,  1], 1.0)]
    push!(element, FieldSet("temperature"))
    # Initial configuration needs to be defined
    push!(element["temperature"], Field(0.0, [0.0, 0.0, 0.0, 0.0]))
    DC2D4NLY(element, integration_points, [])
end
Base.size(equation::DC2D4NLY) = 4

function JuliaFEM.get_residual_vector(equation::DC2D4NLY, ip, time; variation=nothing)
    element = get_element(equation)
    basis = get_basis(element)
    dbasis = grad(basis)
    k = basis("temperature thermal conductivity", ip, time)
    f = basis("temperature load", ip, time)
    T = get_field(basis, "temperature", time, variation)
    f_int = k * dbasis(ip,time)'*dbasis(ip,time) * T
    f_ext = f * basis(ip,time)'
    r = f_int[:] - f_ext[:]
    return r
end
JuliaFEM.has_residual_vector(equation::DC2D4NLY) = true

has_residual_vector (generic function with 2 methods)

In [38]:
function run_simulation_2()
    # create model -- start
    element = Quad4([1, 2, 3, 4])
    fieldset1 = FieldSet("geometry")
    field1 = Field(0.0, Vector[[0.0,0.0], [1.0,0.0], [1.0,1.0], [0.0,1.0]])
    push!(fieldset1, field1)
    fieldset2 = FieldSet("temperature thermal conductivity")
    push!(fieldset2, Field(0.0, 6.0))
    fieldset3 = FieldSet("temperature load")
    push!(fieldset3, Field(0.0, [12.0, 12.0, 12.0, 12.0]))
    fieldset4 = FieldSet("temperature nodal load")
    push!(fieldset4, Field(0.0, [3.0, 3.0, 0.0, 0.0]))  # <-- P is defined here
    push!(element, fieldset1)
    push!(element, fieldset2)
    push!(element, fieldset3)
    push!(element, fieldset4)
    equation = DC2D4NLY(element)
    # create model -- end

    T0 = element["temperature"][1]  # initial temperature field, we need something to "variate"
    la = initialize_local_assembly(equation) # create workspace for local matrices
    T = zeros(4) # create workspace for solution vector
    ΔT = zeros(4) # 
    fd = [1, 2]
    tic()
    # start loops, in principle solve ∂r(u)/∂uΔu = -r(u) and update.
    for i=1:5
        la = initialize_local_assembly(equation,la)  # empty workspace -- every iteration should start with this
        calculate_local_assembly!(la, equation)  # calculate local matrices
        ΔT[fd] = la.stiffness_matrix[fd,fd] \ la.force_vector[fd]  # <-- note sign convention, more on this below
        T = T + ΔT  # add increment to previous value
        # create a new field "similar" to field T0 (i.e., same dimension of field variable with new data)
        new_field = similar(T0, T)
        new_field.time = 1.0
        new_field.increment = i
        push!(element["temperature"], new_field)  # add new field to "temperature" fieldset of element
        # print some convergence information
        @printf("increment %2d, |du| = %8.5f, |r| = %8.3f\n", i, norm(ΔT), norm(b[fd]))
    end
    println("temperature field: ",element["temperature"](Inf).values)
    toc()
end
run_simulation_2()

increment  

0.020793035

1, |du| =  2.82843, |r| =    4.243
increment  2, |du| =  0.00000, |r| =    4.243
increment  3, |du| =  0.00000, |r| =    4.243
increment  4, |du| =  0.00000, |r| =    4.243
increment  5, |du| =  0.00000, |r| =    4.243
temperature field: [2.0000000000000004,2.0000000000000004,0.0,0.0]
elapsed time: 0.020793035 seconds


### Manually assembling local matrices

There might still be situations where all the above methods writing field equations are just not enough. This situation can happen for example when calculating tangent stiffness matrix analytically for a nonlinear problem. In this case all matrices are usually calculated at the same time. Or maybe for educational purposes it's important to show how matrices are actually calculated. Or for debugging. 

Anyway, it's possible to override `calculate_local_assembly!` function for your own equation and after that access all the low level stuff. Here's example how to do that:

In [39]:
using JuliaFEM: LocalAssembly, get_integration_points

function JuliaFEM.calculate_local_assembly!(assembly::LocalAssembly, equation::DC2D4, time::Number=Inf)
    initialize_local_assembly(assembly, equation)  # zero all workspaces
    element = get_element(equation)
    basis = get_basis(element)
    dbasis = grad(basis)
    detJ = det(basis)
    for ip in get_integration_points(equation)
        w = ip.weight * detJ(ip)
        # evaluate fields in integration point
        ρ = basis("density", ip, time)
        k = basis("temperature thermal conductivity", ip, time)
        f = basis("temperature load", ip, time)
        # evaluate basis functions and gradient in integration point
        N = basis(ip, time)
        ∇N = dbasis(ip, time)
        # do assembly
        assembly.mass_matrix += w * ρ*N'*N
        assembly.stiffness_matrix += w * k*∇N'*∇N
        assembly.force_vector += w * (N'*f)[:]
    end
end

function test_local_assembly()
    # create volume element
    element = Quad4([1, 2, 3, 4])
    fieldset1 = FieldSet("geometry", [Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]])])
    fieldset2 = FieldSet("temperature thermal conductivity", [Field(0.0, 6.0)])
    fieldset3 = FieldSet("temperature load", [Field(0.0, [12.0, 12.0, 12.0, 12.0])])
    fieldset4 = FieldSet("density", [Field(0.0, 1.0)])
    push!(element, fieldset1)
    push!(element, fieldset2)
    push!(element, fieldset3)
    push!(element, fieldset4)
    equation = DC2D4(element)
    la = initialize_local_assembly(equation)
    calculate_local_assembly!(la, equation)
    free_dofs = [1, 2]
    @fact la.stiffness_matrix[free_dofs, free_dofs] \ la.force_vector[free_dofs] --> roughly([1.0, 1.0])
end

test_local_assembly()

Success :: (line:-1) :: fact was true
  Expression: la.stiffness_matrix[free_dofs,free_dofs] \ la.force_vector[free_dofs] --> roughly([1.0,1.0])
    Expected: [1.0,1.0]
    Occurred: [1.0000000000000002,1.0000000000000002]

To be continued (stuff below this is a little broken at the moment...)

## Defining own problem

Main objective of "problem" is to take a set of elements and map corresponding field equations to them. In this way we can solve several different fields at the same time, like for example temperature + displacement.

In [34]:
using JuliaFEM: Problem, get_equation, get_dimension

type PlaneHeatProblem <: Problem
    equations :: Array{Equation, 1}
end
PlaneHeatProblem() = PlaneHeatProblem([])

PlaneHeatProblem

In [35]:
JuliaFEM.get_dimension(pr::Type{PlaneHeatProblem}) = 1
JuliaFEM.get_equation(pr::Type{PlaneHeatProblem}, el::Type{Quad4}) = DC2D4
JuliaFEM.get_equation(pr::Type{PlaneHeatProblem}, el::Type{Seg2}) = DC2D2

get_equation (generic function with 6 methods)

Our solution procedure so far is therefore

In [33]:
using JuliaFEM: get_connectivity, set_global_dofs!, get_global_dofs
using JuliaFEM: add_element!, get_equations, get_matrix_dimension, calculate_global_dofs
using JuliaFEM: assign_global_dofs!

# create elements and add necessary properties like connectivity and geometry
el1 = Quad4([2, 3, 4, 5])
fs1geom = FieldSet("geometry")
push!(fs1geom, Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]]))
fs1temp = FieldSet("temperature thermal conductivity")
push!(fs1temp, Field(0.0, 6.0))
push!(el1, fs1geom)
push!(el1, fs1temp)

el2 = Seg2([2, 3])
fs2geom = FieldSet("geometry")
push!(fs2geom, Field(0.0, Vector[[0.0, 0.0], [0.0, 1.0]]))
fs2temp = FieldSet("temperature flux")
push!(fs2temp, Field(1.0, 600.0))
push!(el2, fs2geom)
push!(el2, fs2temp)

problem = PlaneHeatProblem()
push!(problem, el1)
push!(problem, el2)

dofmap = calculate_global_dofs(problem)
assign_global_dofs!(problem, dofmap)

t = 1.0
A = sparse(get_lhs(problem, t)...)
b = sparsevec(get_rhs(problem, t)..., size(A, 1))

25-Oct 23:10:39:DEBUG:root:total dofs: 4


LoadError: LoadError: MethodError: `has_lhs` has no method matching has_lhs(::DC2D4)
while loading In[33], in expression starting on line 30

In [30]:
fdofs = [1, 2]
A[fdofs, fdofs] \ b[fdofs]

LoadError: LoadError: UndefVarError: A not defined
while loading In[30], in expression starting on line 2

We still need to consider Dirichlet boundary conditions:
\begin{align}
u &= u_0 && \text{on } \Gamma_{\mathrm{D}} \\
\end{align}

In [31]:
using JuliaFEM: DirichletProblem

# create elements and add necessary properties like connectivity and geometry
el3 = Seg2([4, 5])
push!(el3, FieldSet("geometry", [Field(0.0, Vector[[0.0, 0.0], [0.0, 1.0]])]))

bc1 = DirichletProblem()
push!(bc1, el3)

1-element Array{JuliaFEM.DirichletEquation,1}:
 JuliaFEM.DBC2D2(JuliaFEM.Seg2([4,5],JuliaFEM.Basis(basis,j),Dict(symbol("reaction force")=>JuliaFEM.FieldSet(symbol("reaction force"),JuliaFEM.Field[]),:geometry=>JuliaFEM.FieldSet(:geometry,JuliaFEM.Field[JuliaFEM.Field{Array{Array{T,1},1}}(0.0,0,Array{T,1}[[0.0,0.0],[0.0,1.0]])]))),[JuliaFEM.IntegrationPoint([-0.5773502691896257],1.0,Dict{Symbol,JuliaFEM.FieldSet}()),JuliaFEM.IntegrationPoint([0.5773502691896257],1.0,Dict{Symbol,JuliaFEM.FieldSet}())],Int64[],fieldval)

In [32]:
assign_global_dofs!(bc1, dofmap)

# integrate and assembly
t = 1.0
A2 = sparse(get_lhs(bc1, t)...)
b2 = sparsevec(get_rhs(bc1, t)...)

LoadError: LoadError: UndefVarError: integrate_lhs not defined
while loading In[32], in expression starting on line 5

Now we have two problems defined, 

In [33]:
full(A)

LoadError: LoadError: UndefVarError: A not defined
while loading In[33], in expression starting on line 1

In [34]:
full(b)

LoadError: LoadError: UndefVarError: b not defined
while loading In[34], in expression starting on line 1

In [35]:
full(A2)

LoadError: LoadError: UndefVarError: A2 not defined
while loading In[35], in expression starting on line 1

In [36]:
full(b2)

LoadError: LoadError: UndefVarError: b2 not defined
while loading In[36], in expression starting on line 1

In [37]:
Atot = [A A2; A2 zeros(A2)]

LoadError: LoadError: UndefVarError: A not defined
while loading In[37], in expression starting on line 1

In [38]:
btot = [b; b2]

LoadError: LoadError: UndefVarError: b not defined
while loading In[38], in expression starting on line 1

Problem is that now are total matrix Atot has zero rows which needs to be removed.

In [39]:
full(Atot)

LoadError: LoadError: UndefVarError: Atot not defined
while loading In[39], in expression starting on line 1

In [40]:
r = unique(rowvals(Atot))
println("Non-zero rows: $r")
xtot = zeros(btot)
F = lufact(Atot[r,r])
s = full(btot[r])
xtot[r] = F \ s
full(xtot)

LoadError: LoadError: UndefVarError: Atot not defined
while loading In[40], in expression starting on line 1

## Developing own solver

Last part. Defining own solver.

- takes a set of problems (typically main problem + boundary problems)
- solves them, updates fields

In [41]:
using JuliaFEM: Solver, get_problems

""" Simple solver for educational purposes. """
type SimpleSolver <: Solver
    problems
end

""" Default initializer. """
function SimpleSolver()
    SimpleSolver(Problem[])
end

"""
Call solver to solve a set of problems.

This is simple serial solver for demonstration purposes. It handles the most
common situation, i.e., some main field problem and it's Dirichlet boundary.
"""
function call(solver::SimpleSolver, t)
    problems = get_problems(solver)
    problem1 = problems[1]
    problem2 = problems[2]

    # calculate order of degrees of freedom in global matrix
    # and set the ordering to problems
    dofmap = calculate_global_dofs(problem1)
    assign_global_dofs!(problem1, dofmap)
    assign_global_dofs!(problem2, dofmap)

    # assemble problem 1
    A1 = sparse(get_lhs(problem1, t)...)
    b1 = sparsevec(get_rhs(problem1, t)..., size(A1, 1))

    # assemble problem 2
    A2 = sparse(get_lhs(problem2, t)...)
    b2 = sparsevec(get_rhs(problem2, t)..., size(A2, 1))
    
    # make one monolithic assembly
    A = [A1 A2; A2' zeros(A2)]
    b = [b1; b2]

    # solve problem
    nz = unique(rowvals(A))
    x = zeros(b)
    x[nz] = lufact(A[nz,nz]) \ full(b[nz])

    # get "problem-wise" solution vectors
    x1 = x[1:length(b1)]
    x2 = x[length(b1)+1:end]

    # check residual
    R1 = A1*x1 - b1
    R2 = A2*x2 - b2
    println("Residual norm: $(norm(R1+R2))")

    # update field for elements in problem 1
    for equation in get_equations(problem1)
        gdofs = get_global_dofs(equation)
        element = get_element(equation)
        field_name = get_unknown_field_name(equation)  # field we are solving
        field = Field(t, full(x1[gdofs])[:])
        push!(element[field_name], field)
    end

    # update field for elements in problem 2
    for equation in get_equations(problem2)
        gdofs = get_global_dofs(equation)
        element = get_element(equation)
        field_name = get_unknown_field_name(equation)
        field = Field(t, full(x2[gdofs]))
        push!(element[field_name], field)
    end
end

call (generic function with 1288 methods)

## Summary

Next we collect all things together and solve Poisson equation in domain $\Omega = [0,1]\times[0,1]$ with some Neumann boundary on $\Gamma_1$ and Dirichlet boundary on $\Gamma_2$.

In [42]:
# Define Problem 1:
# - Field function: Laplace equation Δu=0 in Ω={u∈R²|(x,y)∈[0,1]×[0,1]}
# - Neumann boundary on Γ₁={0<=x<=1, y=0}, ∂u/∂n=600 on Γ₁

el1 = Quad4([1, 2, 3, 4])
push!(el1, FieldSet("geometry", [Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]])]))
push!(el1, FieldSet("temperature thermal conductivity", [Field(0.0, 6.0)]))

el2 = Seg2([1, 2])
push!(el2, FieldSet("geometry", [Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0]])]))

# Boundary load, linear ramp 0 -> 600 at time 0 -> 1
load = FieldSet("temperature flux")
push!(load, Field(0.0, 0.0))
push!(load, Field(1.0, 600.0))
push!(el2, load)

problem1 = PlaneHeatProblem()
push!(problem1, el1)
push!(problem1, el2)

# Define Problem 2:
# - Dirichlet boundary Γ₂={0<=x<=1, y=1}, u=0 on Γ₂

el3 = Seg2([3, 4])
push!(el3, FieldSet("geometry", [Field(0.0, Vector[[1.0, 1.0], [0.0, 1.0]])]))

problem2 = DirichletProblem()
push!(problem2, el3)

# Create a solver for a set of problems
solver = SimpleSolver()
push!(solver, problem1)
push!(solver, problem2)

# Solve problem at time t=1.0 and update fields
call(solver, 1.0)

# Postprocess.
# Interpolate temperature field along boundary of Γ₁ at time t=1.0
xi = linspace([-1.0], [1.0], 5)
X = interpolate(el2, "geometry", xi, 1.0)
T = interpolate(el2, "temperature", xi, 1.0)
println(X)
println(T)

25-Oct 20:27:59:DEBUG:root:total dofs: 4


LoadError: LoadError: MethodError: `has_lhs` has no method matching has_lhs(::DC2D4)
while loading In[42], in expression starting on line 37

In [43]:
@fact mean(T) --> roughly(100.0)

Error :: (line:-1)
  Expression: mean(T) --> roughly(100.0)
  UndefVarError: T not defined
   in anonymous at /home/jukka/.julia/v0.4/FactCheck/src/FactCheck.jl:271
   in do_fact at /home/jukka/.julia/v0.4/FactCheck/src/FactCheck.jl:333
   in include_string at loading.jl:266
   in execute_request_0x535c5df2 at /home/jukka/.julia/v0.4/IJulia/src/execute_request.jl:177
   in eventloop at /home/jukka/.julia/v0.4/IJulia/src/IJulia.jl:141
   in anonymous at task.jl:447

## TODO

- dynamics
- different basis for trial and test (Petrov-Galerkin)

## Frequently asked questions

Any question about data structures, better ideas and improvements are very welcome.